<a href="https://colab.research.google.com/github/MuhammadIrzam447/NewEncodings/blob/main/correctly_classified_Sample_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import pandas as pd
import os
from PIL import Image
import torch
from torchvision import datasets, transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
import torchvision
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import UnidentifiedImageError
from sklearn.metrics import classification_report

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/
!unzip "/content/drive/MyDrive/Attack_on_Multimodal_Learning/Dataset/clippo_test.zip"

Streaming output truncated to the last 5000 lines.
  inflating: clippo/test/shrimp_and_grits/shrimp_and_grits_935_4.png  
  inflating: clippo/test/shrimp_and_grits/shrimp_and_grits_945_3.png  
  inflating: clippo/test/shrimp_and_grits/shrimp_and_grits_945_4.png  
  inflating: clippo/test/shrimp_and_grits/shrimp_and_grits_946_3.png  
  inflating: clippo/test/shrimp_and_grits/shrimp_and_grits_946_4.png  
  inflating: clippo/test/shrimp_and_grits/shrimp_and_grits_949_3.png  
  inflating: clippo/test/shrimp_and_grits/shrimp_and_grits_949_4.png  
  inflating: clippo/test/shrimp_and_grits/shrimp_and_grits_95_3.png  
  inflating: clippo/test/shrimp_and_grits/shrimp_and_grits_95_4.png  
  inflating: clippo/test/shrimp_and_grits/shrimp_and_grits_952_3.png  
  inflating: clippo/test/shrimp_and_grits/shrimp_and_grits_952_4.png  
  inflating: clippo/test/shrimp_and_grits/shrimp_and_grits_960_3.png  
  inflating: clippo/test/shrimp_and_grits/shrimp_and_grits_960_4.png  
  inflating: clippo/test/shr

# Loading Validation Dataset and Preprocessing

In [5]:
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
class ValidationDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.dataset = datasets.ImageFolder(data_dir, transform=val_transform)
        self.classes = sorted(os.listdir(data_dir))
        self.class_lengths = self._compute_class_lengths()
        self.samples = self.dataset.samples  # Store the file paths along with labels


    def _compute_class_lengths(self):
        class_lengths = {cls: 0 for cls in self.classes}

        for cls in self.classes:
            cls_dir = os.path.join(self.data_dir, cls)
            if os.path.isdir(cls_dir):
                class_lengths[cls] = len(os.listdir(cls_dir))

        return class_lengths

    def __getitem__(self, index):
        image, label = self.dataset[index]
        return image, label

    def __len__(self):
        return len(self.dataset)

In [9]:
valPath = "/content/clippo/test"
val_dataset = ValidationDataset(valPath)

In [10]:
batch_size = 32
validation_data_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [11]:
print("Number of samples:", len(val_dataset))
print("Number of classes:", len(val_dataset.classes))

Number of samples: 45432
Number of classes: 101


In [12]:
num_batches = len(validation_data_loader)
print("Number of batches:", num_batches)

Number of batches: 1420


# Validation

In [ ]:
save_dir = '/content/drive/MyDrive/Attack_on_Multimodal_Learning/rk8.04 Resnet_Model'
model_name = "31_model.pth"
load_path = os.path.join(save_dir, model_name)

resnet = torchvision.models.resnet101(pretrained=False)
resnet.fc = nn.Linear(2048, 101) # Choose the number of output classses as per your model
resnet.load_state_dict(torch.load(load_path))

resnet.eval()
resnet.to(device)

In [13]:
# Store misclassified samples
correctly_classified_samples = []

In [16]:
predicted_classes = []
actual_labels = []
correctly_classified_samples = []

with torch.no_grad():
    for i, (images, labels) in enumerate(validation_data_loader):
        # Move the images and labels to the GPU if available
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = resnet(images)

        # Get the predicted labels and probabilities
        _, predicted_labels = torch.max(outputs, 1)
        probabilities = torch.softmax(outputs, dim=1)

        # Store the predicted and true labels
        predicted_classes.extend(predicted_labels.cpu().tolist())
        actual_labels.extend(labels.cpu().tolist())

        # Collect correctly classified samples
        correctly_classified_samples.extend([
            (sample[0], pred.item(), prob[pred.item()].item(), actual.item()) for sample, pred, prob, actual in zip(val_dataset.samples[i * batch_size:(i + 1) * batch_size], predicted_labels.cpu(), probabilities.cpu(), labels.cpu())
            if pred == actual
        ])

In [17]:
len(correctly_classified_samples)

33522

In [18]:
correctly_classified_samples[0]

('/content/clippo/test/apple_pie/apple_pie_103_3.png',
 0,
 0.8242783546447754,
 0)

In [19]:
%cd /content/
with open("fused.txt", "w") as file:
    for item in correctly_classified_samples:
        file.write(str(item) + "\n")

/content


In [20]:
accuracy = accuracy_score(actual_labels, predicted_classes)
precision = precision_score(actual_labels, predicted_classes, average='weighted')
recall = recall_score(actual_labels, predicted_classes, average='weighted')
f1 = f1_score(actual_labels, predicted_classes, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.7378499735868991
Precision: 0.7494257008849378
Recall: 0.7378499735868991
F1-score: 0.7411916344367797


In [21]:
print(classification_report(actual_labels, predicted_classes))

              precision    recall  f1-score   support

           0       0.78      0.74      0.76       468
           1       0.80      0.75      0.77       442
           2       0.82      0.75      0.78       452
           3       0.59      0.65      0.62       444
           4       0.53      0.49      0.51       450
           5       0.77      0.78      0.78       448
           6       0.77      0.75      0.76       448
           7       0.76      0.73      0.74       450
           8       0.73      0.79      0.76       452
           9       0.68      0.71      0.70       428
          10       0.80      0.80      0.80       462
          11       0.76      0.78      0.77       454
          12       0.75      0.79      0.77       460
          13       0.75      0.79      0.77       440
          14       0.81      0.71      0.76       462
          15       0.78      0.75      0.77       454
          16       0.63      0.60      0.62       448
          17       0.81    